In [1]:
import pandas as pd
factors = pd.read_csv('your csv1')
patient_info = pd.read_csv('your csv2')
merged_df1 = pd.merge(factors, patient_info, left_on='patient_about_id', right_on='patient_id', how='inner')
user_info = pd.read_csv('your csv3')
merged_df2 = pd.merge(merged_df1, user_info, left_on='user_input_id', right_on='user_id', how='inner')
notes = pd.read_csv('your csv4')
merged_df3 = pd.merge(merged_df2, notes, left_on='patient_id', right_on='patient_id', how='inner')
df = merged_df3.convert_dtypes()

In [2]:
final_df = df.drop(['external_id', 'id_x', 'symptom_id', 'logged_at_y', 'patient_about_id', 'first_name_y', 'last_name_y', 'date_of_birth_y', 'id_y', 'veteran', 'ethnicity', 'race', 'severity', 'description'], axis = 1)
final_df.dropna(inplace=True)

In [3]:
# final_df.dtypes

user_input_id            string[python]
symptom_date             string[python]
had_symptom                     boolean
factor                   string[python]
category                 string[python]
subcategory              string[python]
logged_at_x              string[python]
patient_id               string[python]
first_name_x             string[python]
last_name_x              string[python]
date_of_birth_x          string[python]
gender                   string[python]
patient_type             string[python]
user_id                  string[python]
phone_number             string[python]
postal_code              string[python]
user_type                string[python]
registered_at            string[python]
country                  string[python]
referral_group           string[python]
city                     string[python]
state                    string[python]
note                     string[python]
additional_notes_date    string[python]
dtype: object

In [ ]:
search_result = final_df.loc[final_df['last_name_x'] == 'Smith', ['factor', 'category', 'first_name_x', 'last_name_x']].drop_duplicates()

print(search_result)

In [ ]:
search_result = final_df[final_df['last_name_x'] == 'Smith']
print(search_result)

In [6]:
import openai
import panel as pn
import os
openai.api_key = "your key here"


In [7]:
def get_completion(prompt, model="gpt-3.5-turbo-16k"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=1600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=1600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [9]:
def get_dataframe(first_name, last_name):
    return final_df[(final_df['last_name_x'] == last_name) & (final_df['first_name_x'] == first_name)]

In [ ]:
import panel as pn
import jupyter_bokeh


# Main Cell
# Prompt the user to enter a value
first_name = input("Please enter your first name: ")
last_name = input("Please enter your last name: ")
# Print the user input
print("You entered:", first_name, last_name)
# print(get_dataframe(first_name, last_name))
info = get_dataframe(first_name, last_name)

pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content': f"""
You are PoPer, an automated service bot to communicate with your patients for a TBI (Traumatic Brain Injuries) patients platform. \
You first greet the patient, {first_name} {last_name}, by the first name. You shall be warm, friendly, and nice. \
You shall always try to provide additional advices based on your own understanding.\
Here are the information of this patient {info.head(n=70).to_string()}
"""
} ]  # accumulate messages
  
print(context)

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Continue!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1300),
)

dashboard